In [1]:
import os
import scipy.io
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torch as t
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torchvision import transforms as tvtsf
from skimage import transform as sktsf
from data.util import *
import numpy as np
import pandas as pd
from trainer import VictimFasterRCNNTrainer
from utils import array_tool as at
from tqdm import tqdm
from torch.autograd import Variable
import matplotlib

matplotlib.use('TkAgg')
import warnings
warnings.filterwarnings('ignore')

In [2]:
import ipdb
from tqdm import tqdm
from utils.config import opt
from data.dataset import Dataset, TestDataset, inverse_normalize
from model import FasterRCNNVGG16
from torch.autograd import Variable
from torch.utils import data as data_
from trainer import FasterRCNNTrainer
from utils import array_tool as at
from utils.vis_tool import visdom_bbox
from utils.eval_tool import eval_detection_voc

In [3]:
import resource

rlimit = resource.getrlimit(resource.RLIMIT_NOFILE)
resource.setrlimit(resource.RLIMIT_NOFILE, (20480, rlimit[1]))
matplotlib.use('TkAgg')

In [4]:
dataset = Dataset(opt)
print('load data')
# dataset = Dataset(opt)

dataloader = data_.DataLoader(dataset, \
                                batch_size=1, \
                                shuffle=True, \
                                # pin_memory=True,
                                num_workers=opt.num_workers)
testset = TestDataset(opt)
test_dataloader = data_.DataLoader(testset,
                                    batch_size=1,
                                    num_workers=opt.test_num_workers,
                                    shuffle=False, \
                                    pin_memory=True
                                    )

load data


In [5]:
faster_rcnn = FasterRCNNVGG16()
print('model construct completed')
trainer = FasterRCNNTrainer(faster_rcnn).cuda()
if opt.load_path:
    trainer.load(opt.load_path)
    print('load pretrained model from %s' % opt.load_path)

Setting up a new session...


model construct completed


In [6]:
trainer.vis.text(dataset.db.label_names, win='labels')
best_map = 0
for epoch in range(opt.epoch):
    trainer.reset_meters()
    for ii, (img, bbox_, label_, scale) in tqdm(enumerate(dataloader)):
        scale = at.scalar(scale)
        img, bbox, label = img.cuda().float(), bbox_.cuda(), label_.cuda()
        img, bbox, label = Variable(img), Variable(bbox), Variable(label)
        trainer.train_step(img, bbox, label, scale)

        if (ii + 1) % opt.plot_every == 0:
            if os.path.exists(opt.debug_file):
                ipdb.set_trace()

            # plot loss
            trainer.vis.plot_many(trainer.get_meter_data())

            # plot groud truth bboxes
            ori_img_ = inverse_normalize(at.tonumpy(img[0]))
            gt_img = visdom_bbox(ori_img_,
                                    at.tonumpy(bbox_[0]),
                                    at.tonumpy(label_[0]))
            trainer.vis.img('gt_img', gt_img)

            # plot predicti bboxes
            _bboxes, _labels, _scores = trainer.faster_rcnn.predict([ori_img_], visualize=True)
            pred_img = visdom_bbox(ori_img_,
                                    at.tonumpy(_bboxes[0]),
                                    at.tonumpy(_labels[0]).reshape(-1),
                                    at.tonumpy(_scores[0]))
            trainer.vis.img('pred_img', pred_img)

            # rpn confusion matrix(meter)
            trainer.vis.text(str(trainer.rpn_cm.value().tolist()), win='rpn_cm')
            # roi confusion matrix
            trainer.vis.img('roi_cm', at.totensor(trainer.roi_cm.conf, False).float())
    eval_result = eval(test_dataloader, faster_rcnn, test_num=opt.test_num)

    if eval_result['map'] > best_map:
        best_map = eval_result['map']
        best_path = trainer.save(best_map=best_map)
    if epoch == 9:
        trainer.load(best_path)
        trainer.faster_rcnn.scale_lr(opt.lr_decay)

    trainer.vis.plot('test_map', eval_result['map'])
    lr_ = trainer.faster_rcnn.optimizer.param_groups[0]['lr']
    log_info = 'lr:{}, map:{},loss:{}'.format(str(lr_),
                                                str(eval_result['map']),
                                                str(trainer.get_meter_data()))
    trainer.vis.log(log_info)
    if epoch == 13:
        break

0it [00:02, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 392.00 MiB (GPU 0; 3.81 GiB total capacity; 2.33 GiB already allocated; 283.44 MiB free; 2.40 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF